In [33]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import window, col, avg, concat, lit, count, round, split, filter, row_number, dense_rank, corr
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, IntegerType, ByteType, BooleanType, FloatType
from pyspark.ml.stat import Correlation
from pyspark.sql.window import Window


sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("DataSourceSinkExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "temp_rgather"
spark.conf.set('temporaryGcsBucket', bucket)

# load and process songs dataset
dataSchema_songs = StructType(
    [StructField("index", IntegerType(), True),
     StructField("track_id", StringType(), True),
     StructField("artists", StringType(), True),
     StructField("album_name", StringType(), True),
     StructField("track_name", StringType(), True),
     StructField("popularity", ByteType(), True),
     StructField("duration_ms", IntegerType(), True),
     StructField("explicit", BooleanType(), True),
     StructField("danceability", FloatType(), True),
     StructField("energy", FloatType(), True),
     StructField("key", ByteType(), True),
     StructField("loudness", FloatType(), True),
     StructField("mode", BooleanType(), True),
     StructField("speechiness", FloatType(), True),
     StructField("acousticness", FloatType(), True),
     StructField("intrumentalness", FloatType(), True),
     StructField("liveness", FloatType(), True),
     StructField("valence", FloatType(), True),
     StructField("tempo", FloatType(), True),
     StructField("time_signature", ByteType(), True),
     StructField("track_genre", StringType(), True)
     ])

df_songs = spark.read.schema(dataSchema_songs).format("csv").option("delimiter",",").option("header", "true") \
       .load("/home/jovyan/data/spotify_songs_full_preproc.csv")
df_songs.printSchema()

# split artists to get lead
df_songs = df_songs.select("*",split("artists","-")[0]).withColumnRenamed("split(artists, -, -1)[0]","lead_artist")

# drop unwanted columns
df_songs = df_songs.drop("artists").drop("track_name").drop("album_name").drop("track_id").drop("index").drop("mode").drop("key").drop("time_signature")

df_songs.show(10)

# get lead artists in genres
df_lead = df_songs.groupby('track_genre','lead_artist').count().orderBy("track_genre")
windowDept = Window.partitionBy("track_genre").orderBy(col("count").desc())
df_lead = df_lead.withColumn("count",row_number().over(windowDept)).filter(col("count") < 2).drop('count')

df_lead.show(10)

# average attributes across genres and add most popular artist
DEC = 3
df_avg = df_songs.groupby('track_genre').agg( \
    round(avg('popularity')).alias("popularity"), \
    round(avg('danceability'),DEC).alias("danceability"),\
    round(avg('energy'),DEC).alias("energy"),\
    round(avg('loudness'),DEC).alias("loudness"),\
    round(avg('speechiness'),DEC).alias("speechiness"),\
    round(avg('acousticness'),DEC).alias("acousticness"),\
    round(avg('intrumentalness'),DEC).alias("intrumentalness"),\
    round(avg('liveness'),DEC).alias("liveness"),\
    round(avg('valence'),DEC).alias("positivity"),\
    round(avg('tempo')).alias("tempo"),\
    avg('duration_ms').cast(IntegerType()).alias("duration_ms"),\
    avg(col('explicit').cast("float")).alias('explicit_perc'),\
    round(corr(col('explicit').cast('float'), "popularity"),3).alias("explicit_pop_corr"),\
    count(lit(1)).alias("amount")\
)

# join dataframes
df_avg = df_avg.join(df_lead,"track_genre","left")
df_avg.show(10)

# load and process artists dataset
dataSchema_artists = StructType(
    [StructField("index", IntegerType(), True),
     StructField("artist", StringType(), True),
     StructField("lead_streams", IntegerType(), True),
     StructField("feat_streams", IntegerType(), True),
     StructField("tracks_amount", IntegerType(), True),
     StructField("one_bil", IntegerType(), True),
     StructField("hundred_mil", IntegerType(), True),
     StructField("last_updated", StringType(), True),
     ])

df_artists = spark.read.schema(dataSchema_artists).format("csv").option("delimiter",",").option("header", "true") \
       .load("/home/jovyan/data/spotify_artists_preproc.csv")
df_artists.printSchema()
df_artists = df_artists.drop("index").drop("feat_streams").drop("one_bil").drop("hundred_mil").drop("last_updated")
df_artists.show(10)

# join dataframes
df_full = df_avg.join(df_artists,df_avg.lead_artist ==  df_artists.artist,"left").drop('artist')

# sort and show
df_full = df_full.sort(df_full.popularity.desc())
df_full.show(114)

df_full.write.format('bigquery') \
  .option('table', 'de2022-362710.musicdataset.genre-characteristics-batch') \
  .mode("overwrite") \
  .save()

print("done")

root
 |-- index: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: byte (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: byte (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: boolean (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- intrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: byte (nullable = true)
 |-- track_genre: string (nullable = true)

+----------+-----------+--------+------------+------+--------+-----------+------------+---------------+--------+-------+-------+-----------+--------

In [4]:
# Stop the spark context
spark.stop()